# Analyze GP# vs closest AA for specific PAs 
# and/or filtered by keys (tnumber, genome, etc.)
make sure to install openpyxl if you don't have it already
```bash
pip install openpyxl
```
reads from:
- db.json
- id_path.txt (contains ids to analyze data from - use '\n' delimiter)
- id_pa.txt (if you want to use the closest PA, which is the default for generating the website dataset)
- find_aas.py output from running it on one or more capsids. Put it in find_aas_output/<PDB_ID>.xlsx (or change the path below)

if you want to specify point arrays, set use_closest_pa to False and change the pa_dict variable. Otherwise, only the best fitting PA will be included. If use_closest_pa is False and pa_dict is empty, all available PAs are sampled and the closest point in the first row is used

writes to:
data_folder/gp_aas.xlsx (additional paths are similar but with automatically created with xl file titles based on filters)

In [19]:
db_file = 'data/db.json'
aa_data_folder_path = 'find_aas_output/'
use_closest_pa = True
id_pa_file = 'data/id_pa.txt'
pa_dict = {} # {'1a34': ['30', '123']} -> for capsid 1a34, only include pa30 and pa123. By default all are included
id_path = 'data/unids.txt'
output_folder_path = './data_folder/'
output_file = 'gp_aas.xlsx'

In [20]:
import json
import openpyxl
from os import listdir
from os.path import isfile, join
mypath = aa_data_folder_path
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

from plot_util import *

with open(db_file, 'r') as f:
    data = json.load(f)

if use_closest_pa:
    with open(id_pa_file, 'r') as f:
        id_pa = {l.split()[0]:l.split()[1] for l in f}
    
def letters(s):
    valids = []
    for character in s:
        if character.isalpha():
            valids.append(character)
    return ''.join(valids)

In [21]:
data['close_aas'] = {}
for f in onlyfiles:
    if '.xlsx' not in f: continue
    xldir = mypath + f
    wb = openpyxl.load_workbook(xldir)
    v = f.split('_', 1)[-1].split('.')[0]
    pas = []
    if v in pa_dict: pas = [f"{v}_pa_{pa}" for pa in pa_dict[v]]
    if len(pas) == 0: pas = wb.sheetnames
    if use_closest_pa and v in id_pa: pas = [f"{v}_pa_{id_pa[v]}"]
    resdict = []
    gp_aas = {}
    for pa_sh in pas:
        sh = wb[pa_sh]
        vitems = []
        for row in sh.iter_rows():
                vitems.append([cell.value for cell in row])
        aas = []
        ds_aas = {}
        for j in range(2, len(vitems)):
            for i in range(len(vitems[0])):
                if vitems[j][i] is not None and '.' in str(vitems[j][i]):
                    d = vitems[j][i]
                    ds_aas[d] = vitems[j][i:i+4]
                    if j == 2:
                        gp_aas[d] = ds_aas[d]
                    if d < 5:
                        aas.append(letters(vitems[j][i+2]))
                        if vitems[j][i+3] != 'N/A': aas.append(letters(vitems[j][i+3]))
        resdict.extend(aas)
    closest_gp_aa = letters(gp_aas[min(gp_aas.keys())][2])
    data['data'][v]['closest_gp_aa'] = closest_gp_aa

In [22]:
with open(id_path, 'r') as f:
    ids = [i.replace('\n', '') for i in f]
id_gp = {v: data['data'][v]['gauge_point'] for v in ids if 'gauge_point' in data['data'][v].keys()}
gplist = list(set(id_gp.values()))
gplist = [str(i) for i in sorted([int(g) for g in gplist])]
aalist = ['HIS','ILE', 'ASP', 'ALA', 'PHE', 'ASN', 'GLY', 'SER', 'GLU', 'TRP', 'MET', 'THR', 'CYS', 'ARG', 'PRO', 'LYS', 'UNK', 'VAL', 'GLN', 'TYR', 'LEU']
gp_aa_dict = {}

for g in gplist:
    gp_aa_dict[g] = {a:0 for a in aalist}
    for i in ids:
        if i in id_gp and id_gp[i] == g and 'closest_gp_aa' in data['data'][i].keys():
            caa = data['data'][i]['closest_gp_aa']
            if caa not in gp_aa_dict[g]: gp_aa_dict[g][caa] = 0
            gp_aa_dict[g][caa] += 1
            
df = pd.DataFrame.from_dict(gp_aa_dict)

In [23]:
# Overwrites output path with excel output
df.to_excel(output_folder_path + output_file)

# Get data by key (ex: tnumber)

In [32]:
# run this cell to view possible keys
for i in list(data['data']['1a34'].keys()): print(i)

In [31]:
# CHANGE KEY TO SPECIFY A DIFFERENT FIELD TO USE
# writes data filtered by the chosen key's possible values to separate xl file for each value
key = 'tnumber'

id_key = {k:v[key] if v[key] != '' else 'NA' for k,v in data['data'].items() if key in v}
key_gp_aa_dict = {}
for g in list(set(id_key.values())):
    key_gp_aa_dict[g] = {}
    id_pool = [i for i in ids if i in id_key and id_key[i] == g]
    for gp in gplist:
        key_gp_aa_dict[g][gp] = {a:0 for a in aalist}
        for i in id_pool:
            if i in id_gp and id_gp[i] == gp and 'closest_gp_aa' in data['data'][i]:
                caa = data['data'][i]['closest_gp_aa']
                if caa not in key_gp_aa_dict[g][gp]: key_gp_aa_dict[g][gp][caa] = 0
                key_gp_aa_dict[g][gp][caa] += 1 
# might want to check to make sure you don't have too many outputs, as each one makes a separate xlsx file
# for k in key_gp_aa_dict.keys(): print(k)

In [29]:
for k,v in key_gp_aa_dict.items():
    d = pd.DataFrame.from_dict(v)
    d.to_excel(f"{output_folder}{key}_{k}_{output_file}")